In [ ]:
import os
from pathlib import Path

In [ ]:
path_current = os.getcwd()
path_main = str(Path(path_current).parent)

### To main directory

In [ ]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [ ]:
# load libraries and palette
%run __libraries.ipynb

In [ ]:
# load functions
%run __functions.ipynb

In [ ]:
# load regions data
path_regions_data = path_main + '/files'
path_change(path_regions_data)

%run regions_data.ipynb

In [ ]:
# start stopwatch
start = stopwatch_start()

### Variables

In [ ]:
path_files = path_main + '/files'

In [ ]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [ ]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

In [ ]:
prices_2021 = economics_data['prices_2021'] \
                if 'prices_2021' in economics_data.keys() else None

### Back to notebook directory

In [ ]:
# back to current path
os.chdir(path_current)

# <font color='#2C8B6D'>II. Цены, инфляция</font>

# Section I. Потребительские цены

## Средние потребительские цены

### Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2021 г.)

In [ ]:
if prices_2021 is None:
    %run CP-consumers-2021.ipynb

### 9.2. Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2022 г.)

### 9.3. Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2023 г.)

*Источник: https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_10-2023.xlsx*  
*Единицы измерения: *

*Примечания:*  

#### 2024 year link formula

In [ ]:
for i in range(1,13):
    link = f'https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_{str(i).zfill(2)}-2024.xlsx'
    link_next = f'https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_{str(i+1).zfill(2)}-2024.xlsx'
    response = requests.get(link)
    response_next = requests.get(link_next)
    condition = ((response.status_code == 200) & (response_next.status_code == 404))
    if condition:
        pci_counsumers_link = link

In [ ]:
pci_counsumers_link

##### Extract Data 

In [ ]:
# read excel file
prices_2022_2023_data = pd.ExcelFile(pci_counsumers_link)

In [ ]:
# assign sheet names
prices_2022_2023_data_sheets = prices_2022_2023_data.sheet_names

In [ ]:
# remove 'Содержание'
prices_2022_2023_data_sheets = prices_2022_2023_data_sheets[1:]

In [ ]:
print(prices_2022_2023_data_sheets)

In [ ]:
prices_2022_2023_raw_dict = {}
for sheet in prices_2022_2023_data_sheets:
    prices_2022_2023_raw_dict['prices_'+ sheet[:2] + '_' + sheet[3:7]] = \
        prices_2022_2023_data.parse(sheet)

In [ ]:
prices_2022_2023_raw_dict.keys()

In [ ]:
# find number of the month of last year in Rosstat data
full_months = len(prices_2022_2023_data_sheets) // 12
month_number = len(prices_2022_2023_data_sheets) - full_months*12

In [ ]:
months_2022_2023 = months_list*full_months + months_list[:month_number]

In [ ]:
print(months_2022_2023)

In [ ]:
prices_2022_2023_dict = {}
zip_ = zip(prices_2022_2023_raw_dict.keys(), months_2022_2023, prices_2022_2023_data_sheets)
for key, month, sheet in zip_:
    prices_2022_2023_dict['prices_' + month.lower() + '_' + sheet[-5:-1]] = \
        transform_prices_9_6(prices_2022_2023_raw_dict[key], month, sheet[-5:-1])

##### Concat DataFrames

In [ ]:
prices_2022 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2022'].columns)
prices_2023 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2023'].columns)
prices_2024 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2024'].columns)
for key in prices_2022_2023_dict.keys():
    if key[-4:] == '2022':
        prices_2022 = pd.merge(
            prices_2022.T, prices_2022_2023_dict[key].T,
            on=prices_2022.columns)
        prices_2022.index = prices_2022['key_0']
        prices_2022.index.name = None
        prices_2022 = prices_2022.drop('key_0', axis=1)
        prices_2022 = prices_2022.T
    elif key[-4:] == '2023':
        prices_2023 = pd.merge(
            prices_2023.T, prices_2022_2023_dict[key].T,
            on=prices_2023.columns)
        prices_2023.index = prices_2023['key_0']
        prices_2023.index.name = None
        prices_2023 = prices_2023.drop('key_0', axis=1)
        prices_2023 = prices_2023.T
    elif key[-4:] == '2024':
        prices_2024 = pd.merge(
            prices_2024.T, prices_2022_2023_dict[key].T,
            on=prices_2024.columns)
        prices_2024.index = prices_2024['key_0']
        prices_2024.index.name = None
        prices_2024 = prices_2024.drop('key_0', axis=1)
        prices_2024 = prices_2024.T

In [ ]:
prices_2022.columns = [i.lower() for i in prices_2022.columns]
prices_2023.columns = [i.lower() for i in prices_2023.columns]
prices_2024.columns = [i.lower() for i in prices_2024.columns]

In [ ]:
prices_2022 = prices_2022.astype(float)
prices_2023 = prices_2023.astype(float)
prices_2024 = prices_2024.astype(float)

In [ ]:
prices_2022 = prices_2022.round(2)
prices_2023 = prices_2023.round(2)
prices_2024 = prices_2024.round(2)

In [ ]:
prices_2022

In [ ]:
prices_2023

In [ ]:
prices_2024

### Combibnig Prices 

In [ ]:
prices = (prices_2021.T
          .join(prices_2022.T, how='outer')
          .join(prices_2023.T, how='outer')
          .join(prices_2024.T, how='outer')).T.copy()

In [ ]:
len(prices.T)

In [ ]:
prices.head()

In [ ]:
prices.tail()

### Growth since January 2021

In [ ]:
prices_growth = pd.DataFrame()
for column in prices.columns:
    prices_growth = pd.concat(
        [prices_growth, normalized_by_first(prices[column], return_type='df')],
        axis=1)

In [ ]:
prices_growth.head()

#### <font color='#820A22'> Export Dataset to Excel and CSV

In [ ]:
saveit_excel(
    data=prices,
    filename='prices-2021-2024-consumers-food',
    path=path_excel_csv,
    sheet='Цены')

In [ ]:
saveit_csv(
    data=prices, 
    filename='prices-2021-2024-consumers-food',
    path=path_excel_csv)

In [ ]:
saveit_excel(
    data=prices_growth,
    filename='prices-growth-2021-2024-consumers-food',
    path=path_excel_csv,
    sheet='Цены')

In [ ]:
saveit_csv(
    data=prices_growth, 
    filename='prices-growth-2021-2024-consumers-food',
    path=path_excel_csv)

### Save Datasets

In [ ]:
economics_data['prices_consumers_food'] = prices

In [ ]:
economics_data['prices_consumers_food_growth'] = prices_growth

In [ ]:
economics_data['prices_2021_consumers_food'] = prices_2021

In [ ]:
economics_data['prices_2022_consumers_food'] = prices_2022

In [ ]:
economics_data['prices_2023_consumers_food'] = prices_2023

In [ ]:
economics_data['prices_2024_consumers_food'] = prices_2024

In [ ]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

### Execution time

In [ ]:
stopwatch_stop(start)